In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

import scipy.constants as cst
from lammpsrun import LAMMPS, Prism

from ase.visualize import view
from ase.io.trajectory import Trajectory
from ase.io import write
from ase.build import *

homepath = "D:\\Cloud Storage\\GitRepositories\\LAMMPS_Simulation\\"
homepath

Populating the interactive namespace from numpy and matplotlib


'D:\\Cloud Storage\\GitRepositories\\LAMMPS_Simulation\\'

Per Prof. Nic Harrison,
$$ SE = \frac{E_{slab} - E_{bulk}}{2A}$$

In [2]:
Fe_slab_reaxff_params = {
    # Initialization
    "units": "real",
    "atom_style": "charge",
    "mass": ["1 55.845"],
    
    # Forcefield definition
    "pair_style": "reax/c NULL safezone 16",
    "pair_coeff": ['* * ' + 'ffield.reax.Fe_O_C_H' + ' Fe'],
    "neighbor": "2.0 bin",
    "neighbor_modify": "delay 10 check yes",
    
    # Run and Minimization
    "minimize": "1e-25 1e-25 10000 10000",
    "min_style": "cg",
    "timestep": 0.1,
    "fix": ["1 all box/relax xy 0.0 vmax 0.001",
            "2 all qeq/reax 1 0.0 10.0 1e-6 reax/c"]
}

Fe_bulk_reaxff_params = {
    # Initialization
    "units": "real",
    "atom_style": "charge",
    "mass": ["1 55.845"],
    
    # Forcefield definition
    "pair_style": "reax/c NULL safezone 16",
    "pair_coeff": ['* * ' + 'ffield.reax.Fe_O_C_H' + ' Fe'],
    "neighbor": "2.0 bin",
    "neighbor_modify": "delay 10 check yes",
    
    # Run and Minimization
    "minimize": "1e-25 1e-25 10000 10000",
    "min_style": "cg",
    "timestep": 0.1,
    "fix": ["1 all box/relax iso 0.0 vmax 0.001",
            "2 all qeq/reax 1 0.0 10.0 1e-6 reax/c"]
}

## Bulk Fe

In [3]:
# Creating simulation box
Fe_bulk = bulk('Fe', 'bcc')

# Setting up simulation input
Fe_bulk_calc = LAMMPS(tmp_dir = homepath + "temp", 
                      parameters = Fe_bulk_reaxff_params,
                      keep_tmp_files=True, no_data_file=True)
Fe_bulk.set_calculator(Fe_bulk_calc)
Fe_bulk_calc.atoms = Fe_bulk.copy()
Fe_bulk_calc.prism = Prism(Fe_bulk_calc.atoms.get_cell())

# Writing input file
Fe_bulk_calc.write_lammps_in(lammps_in = homepath + "surface_Energy\\fe_bulk_reaxff_minimize.in",
                             lammps_trj = homepath + "surface_Energy\\fe_bulk_reaxff_minimize.lammpstrj")


In [8]:
# Running calculations
!lmp_serial -in surface_Energy\fe_bulk_reaxff_minimize.in -log surface_Energy\fe_bulk_reaxff_minimize.log

LAMMPS (16 Aug 2018)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:87)
  using 1 OpenMP thread(s) per MPI task
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:87)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 1 1 1
Created triclinic box = (0 0 0) to (2.48549 2.34335 2.0294) with tilt (-0.828498 0.828498 1.17167)
  1 by 1 by 1 MPI processor grid
Created 1 atoms
  Time spent = 0.000496864 secs
Reading potential file ffield.reax.Fe_O_C_H with DATE: 2011-02-18
Neighbor list info ...
  update every 1 steps, delay 0 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 12
  ghost atom cutoff = 12
  binsize = 6, bins = 1 1 1
  2 neighbor lists, perpetual/occasional/extra = 2 0 0
  (1) pair reax/c, perpetual
      attributes: half, newton off, ghost
      pair build: half/bin/newtoff/ghost
      stencil: half/ghost/bin/3d/newtoff
      bin: standard
  (2) fix qeq/reax, perp

In [4]:
# Reading output
Fe_bulk_calc.read_lammps_log(lammps_log = homepath + "surface_Energy\\fe_bulk_reaxff_minimize.log")
Fe_bulk_data = Fe_bulk_calc.thermo_content[-1]

# bcc(110) Fe

In [5]:
# Creating simulation box
Fe_bcc110 = bcc110('Fe', size=(1,1,128), vacuum=10.0)

# Setting up simulation input
Fe_bcc110_calc = LAMMPS(tmp_dir = homepath + "temp", 
                      parameters = Fe_slab_reaxff_params,
                      keep_tmp_files=True, no_data_file=True)
Fe_bcc110.set_calculator(Fe_bcc110_calc)
Fe_bcc110_calc.atoms = Fe_bcc110.copy()
Fe_bcc110_calc.prism = Prism(Fe_bcc110_calc.atoms.get_cell())

# Writing input file
Fe_bcc110_calc.write_lammps_in(lammps_in = homepath + "surface_Energy\\fe_bcc110_reaxff_minimize.in",
                               lammps_trj = homepath + "surface_Energy\\fe_bcc110_reaxff_minimize.lammpstrj")


In [14]:
# Running calculations
!lmp_serial -in surface_Energy\fe_bcc110_reaxff_minimize.in -log surface_Energy\fe_bcc110_reaxff_minimize.log

LAMMPS (16 Aug 2018)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:87)
  using 1 OpenMP thread(s) per MPI task
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:87)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 1 1 1
Created triclinic box = (0 0 0) to (2.87 2.0294 277.733) with tilt (-1.435 0 0)
  1 by 1 by 1 MPI processor grid
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0.000496864 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 a

    1021            0   -26483.848    697.16621   -37076.311   -42344.383   -30.851239   0.06337432 -0.033283009 2.0166024e-008            0   -13188.504   -13188.504    1478.6516         2.87    2.0293965    253.87332      128 
    1022            0   -26483.845    697.87301   -37076.308   -42344.381   -30.844934  -0.36777393 -0.033365794 9.6347058e-009            0   -13188.504   -13188.504    1478.6516         2.87    2.0293965    253.87332      128 
    1023            0   -26483.678    698.60956   -37076.191   -42344.302    -30.54198 -0.054777173 -0.037096341 -8.4131534e-008            0   -13188.504   -13188.504    1478.6516         2.87    2.0293965    253.87332      128 
    1024            0   -26483.675    699.42052   -37076.189     -42344.3   -30.536488   0.34267092 -0.037144111 -7.7152203e-008            0   -13188.504   -13188.504    1478.6516         2.87    2.0293965    253.87332      128 
    1025            0   -26483.548    700.13824   -37076.099    -42344.24   -30.30

    1520            0   -26466.893    1028.9917   -37064.346   -42336.335 0.0011449402 -4.1443318e-007 -0.0023500854 8.3486955e-010            0   -13188.504   -13188.504    1478.6516         2.87    2.0293965    253.87332      128 
    1521            0   -26466.893    1029.6509   -37064.346   -42336.335 0.0011596026 0.00013660897 -0.0023374383 5.1884465e-010            0   -13188.504   -13188.504    1478.6516         2.87    2.0293965    253.87332      128 
    1522            0   -26466.893    1030.3458   -37064.346   -42336.335 0.0011662777 -1.0249102e-006 -0.0023316403 6.7667256e-010            0   -13188.504   -13188.504    1478.6516         2.87    2.0293965    253.87332      128 
    1523            0   -26466.893    1031.0566   -37064.346   -42336.335 0.0011851975 -2.4186022e-005 -0.0023131059 5.3817173e-010            0   -13188.504   -13188.504    1478.6516         2.87    2.0293965    253.87332      128 
    1524            0   -26466.893    1031.7366   -37064.346   -42336.

    2520            0   -26466.894    1668.5303   -37064.346   -42336.336 5.8197964e-006 -1.3489768e-008 -1.1876349e-007 -2.2042244e-010            0   -13188.504   -13188.504    1478.6516         2.87    2.0293965    253.87332      128 
    2521            0   -26466.894    1669.1429   -37064.346   -42336.336 5.8564028e-006 1.323055e-008 -1.1864542e-007 -9.0493316e-010            0   -13188.504   -13188.504    1478.6516         2.87    2.0293965    253.87332      128 
    2522            0   -26466.894     1669.751   -37064.346   -42336.336 5.7377988e-006 1.1294556e-008 -1.1800513e-007 5.9279047e-010            0   -13188.504   -13188.504    1478.6516         2.87    2.0293965    253.87332      128 
    2523            0   -26466.894    1670.3536   -37064.346   -42336.336 5.8333595e-006 -4.0561766e-009 -1.1904859e-007 -4.4262375e-010            0   -13188.504   -13188.504    1478.6516         2.87    2.0293965    253.87332      128 
    2524            0   -26466.894    1670.9706   -3

In [6]:
# Reading output
Fe_bcc110_calc.read_lammps_log(lammps_log = homepath + "surface_Energy\\fe_bcc110_reaxff_minimize.log")
Fe_bcc110_data = Fe_bcc110_calc.thermo_content[-1]

In [14]:
Fe_bcc110_data = Fe_bcc110_calc.thermo_content[-1]

# Surface energy in Kcal/(mol A^2)
Fe_bcc110_SE_real = (Fe_bcc110_data['pe']/Fe_bcc110_data['atoms'] - Fe_bulk_data['pe']/Fe_bulk_data['atoms'])/(2*(Fe_bcc110_data['vol']/Fe_bcc110_data['lz']/Fe_bcc110_data['atoms']))

# Converting to J/m^2
Fe_bcc110_SE_SI = Fe_bcc110_SE_real * 4184 / cst.Avogadro * 10**20
print(Fe_bcc110_SE_SI)

1.7443148875538246


# bcc(100) Fe

In [7]:
# Creating simulation box
Fe_bcc100 = bcc100('Fe', size=(1,1,128), vacuum=10.0)

# Setting up simulation input
Fe_bcc100_calc = LAMMPS(tmp_dir = homepath + "temp", 
                      parameters = Fe_slab_reaxff_params,
                      keep_tmp_files=True, no_data_file=True, always_triclinic=True)
Fe_bcc100.set_calculator(Fe_bcc100_calc)
Fe_bcc100_calc.atoms = Fe_bcc100.copy()
Fe_bcc100_calc.prism = Prism(Fe_bcc100_calc.atoms.get_cell())

# Writing input file
Fe_bcc100_calc.write_lammps_in(lammps_in = homepath + "surface_Energy\\fe_bcc100_reaxff_minimize.in",
                               lammps_trj = homepath + "surface_Energy\\fe_bcc100_reaxff_minimize.lammpstrj")

In [19]:
# Running calculations
!lmp_serial -in surface_Energy\fe_bcc100_reaxff_minimize.in -log surface_Energy\fe_bcc100_reaxff_minimize.log

LAMMPS (16 Aug 2018)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:87)
  using 1 OpenMP thread(s) per MPI task
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:87)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 1 1 1
Created triclinic box = (0 0 0) to (2.87 2.87 202.245) with tilt (0 0 0)
  1 by 1 by 1 MPI processor grid
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0.000497103 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  

In [8]:
# Reading output
Fe_bcc100_calc.read_lammps_log(lammps_log = homepath + "surface_Energy\\fe_bcc100_reaxff_minimize.log")
Fe_bcc100_data = Fe_bcc100_calc.thermo_content[-1]

In [13]:
Fe_bcc100_data = Fe_bcc100_calc.thermo_content[-1]

# Surface energy in Kcal/(mol A^2)
Fe_bcc100_SE_real = (Fe_bcc100_data['pe']/Fe_bcc100_data['atoms'] - Fe_bulk_data['pe']/Fe_bulk_data['atoms'])/(2*(Fe_bcc100_data['vol']/Fe_bcc100_data['lz']/Fe_bcc100_data['atoms']))

# Converting to J/m^2
Fe_bcc100_SE_SI = Fe_bcc100_SE_real * 4184 / cst.Avogadro * 10**20
print(Fe_bcc100_SE_SI)

2.0692676658333244


# bcc(111) Fe

In [9]:
# Creating simulation box
Fe_bcc111 = bcc111('Fe', size=(1,1,128), vacuum=10.0)

# Setting up simulation input
Fe_bcc111_calc = LAMMPS(tmp_dir = homepath + "temp", 
                      parameters = Fe_slab_reaxff_params,
                      keep_tmp_files=True, no_data_file=True)
Fe_bcc111.set_calculator(Fe_bcc111_calc)
Fe_bcc111_calc.atoms = Fe_bcc111.copy()
Fe_bcc111_calc.prism = Prism(Fe_bcc111_calc.atoms.get_cell())

# Writing input file
Fe_bcc111_calc.write_lammps_in(lammps_in = homepath + "surface_Energy\\fe_bcc111_reaxff_minimize.in",
                               lammps_trj = homepath + "surface_Energy\\fe_bcc111_reaxff_minimize.lammpstrj")

In [23]:
# Running calculations
!lmp_serial -in surface_Energy\fe_bcc111_reaxff_minimize.in -log surface_Energy\fe_bcc111_reaxff_minimize.log

LAMMPS (16 Aug 2018)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:87)
  using 1 OpenMP thread(s) per MPI task
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:87)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 1 1 1
Created triclinic box = (0 0 0) to (4.05879 3.51502 125.219) with tilt (-2.0294 0 0)
  1 by 1 by 1 MPI processor grid
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0.000495911 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Create

In [10]:
# Reading output
Fe_bcc111_calc.read_lammps_log(lammps_log = homepath + "surface_Energy\\fe_bcc111_reaxff_minimize.log")

In [39]:
Fe_bcc111_calc.thermo_content[-1]

{'step': 167.0,
 'temp': 0.0,
 'press': -28338.685,
 'cpu': 59.05328,
 'pxx': -41163.782,
 'pyy': -41161.908,
 'pzz': -2690.3641,
 'pxy': 7.5989013,
 'pxz': -19.164946,
 'pyz': -0.23352726,
 'ke': 0.0,
 'pe': -13107.84,
 'etotal': -13107.84,
 'vol': 1473.8422,
 'lx': 4.0587929,
 'ly': 3.5150178,
 'lz': 103.30624,
 'atoms': 128.0}

In [12]:
Fe_bcc111_data = Fe_bcc111_calc.thermo_content[-1]

# Surface energy in Kcal/(mol A^2)
Fe_bcc111_SE_real = (Fe_bcc111_data['pe']/Fe_bcc111_data['atoms'] - Fe_bulk_data['pe']/Fe_bulk_data['atoms'])/(2*(Fe_bcc111_data['vol']/Fe_bcc111_data['lz']/Fe_bcc111_data['atoms']))

# Converting to J/m^2
Fe_bcc111_SE_SI = Fe_bcc111_SE_real * 4184 / cst.Avogadro * 10**20
print(Fe_bcc111_SE_SI)

2.6762248435373914


# etc

In [ ]:

Fe_var_reaxff_thermo_contents = []
Fe_var_reaxff_thermo_contents_label = []

for thickness in range(2, 60, 4):
    Fe_var_reaxff_thermo_contents_label.append(thickness)
    Fe_var_reaxff_slab = bcc110('Fe', size=(1,1,thickness), vacuum=10.0)
    
    Fe_var_reaxff_slab_calc = LAMMPS(tmp_dir = homepath + "temp", 
                              parameters = fe_slab_lammps_params,
                              keep_tmp_files=True, no_data_file=True)
    
    Fe_slab.set_calculator(Fe_var_reaxff_slab_calc)
    Fe_var_reaxff_slab_calc.atoms = Fe_var_reaxff_slab.copy()
    Fe_var_reaxff_slab_calc.prism = Prism(Fe_var_reaxff_slab_calc.atoms.get_cell())
    
    Fe_var_reaxff_slab_calc.write_lammps_in(lammps_in = homepath + "input\\fe_var_slab_reaxff_minimize.in",
                                     lammps_trj = homepath + "output\\fe_var_slab_reaxff_minimize.lammpstrj")
    
    !lmp_serial -in input\fe_var_slab_reaxff_minimize.in -log output\fe_var_slab_reaxff_minimize.log
    
    Fe_var_reaxff_slab_calc.read_lammps_log(lammps_log = homepath + "output\\fe_var_slab_reaxff_minimize.log")
    Fe_var_reaxff_thermo_contents.append(Fe_var_reaxff_slab_calc.thermo_content)